In [7]:
!pip install -q --no-deps --upgrade pyarrow==14.0.1 transformers==4.40.0 tokenizers==0.19.1 datasets==2.14.6 peft==0.11.1 accelerate==0.27.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 51.0 MB/s eta 0:00:00:00:0100:01


In [1]:
import torch

print("GPU Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    print("GPU Memory:", torch.cuda.get_device_properties(0).total_memory / 1e9, "GB")


GPU Available: True
GPU Name: Tesla T4
GPU Memory: 15.828320256 GB


In [2]:
import json
from datasets import Dataset

# Your Bengali empathetic conversation data
train_data = [
    {"text": "Below is a conversation between a user and an empathetic assistant.\n\n### User:\nআমি খুবই দুঃখী বোধ করছি, কিছুই ভালো লাগছে না।\n\n### Assistant:\nআমি বুঝতে পারছি আপনি খুব কঠিন সময়ের মধ্য দিয়ে যাচ্ছেন। এভাবে অনুভব করা স্বাভাবিক, কিন্তু আপনি একা নন।"},
    {"text": "Below is a conversation between a user and an empathetic assistant.\n\n### User:\nআমার পড়াশোনা ভালো যাচ্ছে না, আমি নিজেকে ব্যর্থ মনে করছি।\n\n### Assistant:\nনিজেকে ব্যর্থ মনে হওয়া খুব কষ্টের অনুভূতি। কিন্তু একটি খারাপ সময় আপনার সব দক্ষতা নষ্ট করে না।"},
    {"text": "Below is a conversation between a user and an empathetic assistant.\n\n### User:\nআমার পরিবার আমাকে বুঝে না বলে মনে হয়।\n\n### Assistant:\nপরিবারের দ্বারা না বোঝা হওয়া সত্যিই কষ্টদায়ক। আপনার অনুভূতিগুলো শান্তভাবে তুলে ধরার চেষ্টা করুন।"},
    {"text": "Below is a conversation between a user and an empathetic assistant.\n\n### User:\nআমার কোনো বন্ধু নেই মনে হয়, আমি একা বোধ করি।\n\n### Assistant:\nএকাকিত্ব খুব ভারী অনুভূতি হতে পারে। ধীরে ধীরে মানুষদের সাথে ছোট ছোট আলাপ শুরু করুন।"},
    {"text": "Below is a conversation between a user and an empathetic assistant.\n\n### User:\nআমি সব সময় ভবিষ্যৎ নিয়ে চিন্তায় থাকি।\n\n### Assistant:\nভবিষ্যৎ নিয়ে দুশ্চিন্তা করা খুব স্বাভাবিক। বর্তমানে আপনি কী নিয়ন্ত্রণ করতে পারেন সেদিকে ফোকাস করুন।"},
    {"text": "Below is a conversation between a user and an empathetic assistant.\n\n### User:\nআমার কাজের প্রতি কোন আগ্রহ নেই, সবকিছু ক্লান্তিকর লাগে।\n\n### Assistant:\nদীর্ঘদিন ধরে ক্লান্তি মানসিক স্বাস্থ্যের লক্ষণ হতে পারে। নিজের প্রতি সহানুভূতিশীল হোন।"},
    {"text": "Below is a conversation between a user and an empathetic assistant.\n\n### User:\nআমি ছোট ছোট ভুল করলেই নিজেকে ঘৃণা করি।\n\n### Assistant:\nসবাই ভুল করে। ভুল থেকে শিখে এগিয়ে যাওয়ার চেষ্টা করুন।"},
    {"text": "Below is a conversation between a user and an empathetic assistant.\n\n### User:\nসবাই যেন আমার থেকে অনেক এগিয়ে গেছে, আমি পিছিয়ে পড়েছি।\n\n### Assistant:\nপ্রত্যেকের যাত্রা আলাদা। নিজের ছোট অগ্রগতি চিহ্নিত করার চেষ্টা করুন।"},
    {"text": "Below is a conversation between a user and an empathetic assistant.\n\n### User:\nআমি আমার ক্যারিয়ার নিয়ে খুবই অনিশ্চিত বোধ করছি।\n\n### Assistant:\nক্যারিয়ার নিয়ে অনিশ্চয়তা অনেকের জীবনেই আসে। ছোট পদক্ষেপ নিয়ে এগিয়ে যান।"},
    {"text": "Below is a conversation between a user and an empathetic assistant.\n\n### User:\nআমি আমার ভুলের জন্য নিজেকে ক্ষমা করতে পারব বলে মনে হয় না।\n\n### Assistant:\nগভীর অনুশোচনা ভারী হতে পারে। নিজেকে ধীরে ধীরে ক্ষমা করা একটি প্রক্রিয়া।"},
]

val_data = [
    {"text": "Below is a conversation between a user and an empathetic assistant.\n\n### User:\nআমার সাথে কেউ খোলামেলা কথা বলতে চায় না।\n\n### Assistant:\nএমন অনুভূতি খুব একাকিত্বের জন্ম দিতে পারে। এমন কাউকে খুঁজুন যিনি শুনবেন।"},
    {"text": "Below is a conversation between a user and an empathetic assistant.\n\n### User:\nমানুষ আমাকে সবসময় ভুল বোঝে।\n\n### Assistant:\nভুল বোঝাবুঝি হতাশাজনক। আপনি আপনার সত্য জানেন এবং সেটাই গুরুত্বপূর্ণ।"},
]

# Create HuggingFace Datasets
train_ds = Dataset.from_list(train_data)
val_ds = Dataset.from_list(val_data)

print(f"✓ Training examples: {len(train_ds)}")
print(f"✓ Validation examples: {len(val_ds)}")
print(f"✓ Sample: {train_data[0]}")


✓ Training examples: 10
✓ Validation examples: 2
✓ Sample: {'text': 'Below is a conversation between a user and an empathetic assistant.\n\n### User:\nআমি খুবই দুঃখী বোধ করছি, কিছুই ভালো লাগছে না।\n\n### Assistant:\nআমি বুঝতে পারছি আপনি খুব কঠিন সময়ের মধ্য দিয়ে যাচ্ছেন। এভাবে অনুভব করা স্বাভাবিক, কিন্তু আপনি একা নন।'}


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model

print("✓ Loading Model...")
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# 1. Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# 2. Model (float16 to fit T4 GPU easily)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# 3. LoRA Configuration
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# 4. Apply LoRA
model = get_peft_model(model, config)
model.print_trainable_parameters()


2026-01-01 17:05:41.949929: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767287142.115074      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767287142.161152      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767287142.543277      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767287142.543315      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767287142.543317      55 computation_placer.cc:177] computation placer alr

✓ Loading Model...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

trainable params: 4,505,600 || all params: 1,104,553,984 || trainable%: 0.4079


In [4]:
print("✓ Tokenizing data...")

def tokenize_function(examples):
    """Tokenize text and set labels for causal language modeling"""
    tokenized = tokenizer(
        examples["text"],
        padding="max_length",
        max_length=256,          # Smaller sequence length to fit in memory
        truncation=False,        # Do NOT truncate (requirement)
        return_tensors=None
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# Tokenize train and validation datasets
tokenized_train = train_ds.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized_val = val_ds.map(tokenize_function, batched=True, remove_columns=["text"])

print(f"✓ Tokenized Train: {len(tokenized_train)} examples")
print(f"✓ Tokenized Val: {len(tokenized_val)} examples")
print(f"✓ First sample keys: {list(tokenized_train[0].keys())}")
print(f"✓ First sample shape: input_ids={len(tokenized_train[0]['input_ids'])}, labels={len(tokenized_train[0]['labels'])}")


✓ Tokenizing data...


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

✓ Tokenized Train: 10 examples
✓ Tokenized Val: 2 examples
✓ First sample keys: ['input_ids', 'attention_mask', 'labels']
✓ First sample shape: input_ids=256, labels=256


In [5]:
from transformers import TrainingArguments, Trainer

print("✓ Setting up Training...")

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,                # 3 epochs
    per_device_train_batch_size=1,     # Small batch size for safety
    gradient_accumulation_steps=4,     # Accumulate gradients to simulate larger batch
    learning_rate=2e-4,                # Standard LoRA learning rate
    logging_steps=1,
    fp16=False,                        # Disable mixed precision to avoid conflict
    save_strategy="no",                # Don't save checkpoints to save disk space
    report_to=[],                      # Disable wandb/logging integrations
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

print("✓ Starting Training...")
trainer.train()
print("✓ Training Finished!")


✓ Setting up Training...
✓ Starting Training...


Step,Training Loss
1,5.001500
2,4.061000
3,4.075300
4,3.108200
5,2.945600
6,2.579300
7,2.456600
8,2.015600
9,1.854600


✓ Training Finished!


In [6]:
import math

print("✓ Running evaluation on validation set...")
eval_results = trainer.evaluate()

val_loss = eval_results["eval_loss"]
perplexity = math.exp(val_loss)

print("Validation Loss:", val_loss)
print("Perplexity:", perplexity)


✓ Running evaluation on validation set...


Validation Loss: 2.39481520652771
Perplexity: 10.966171394939236
